In [14]:
from openai import OpenAI
import os
import re
import base64
import requests
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from IPython.display import FileLink, display
from dotenv import load_dotenv
from random import shuffle, randint, choice, random
from collections import Counter
# Load API key
_ = load_dotenv("../comm4190_F25/01_Introduction_and_setup/.env")
client = OpenAI()

In [2]:
# prompt for brainstorming ideas
SYSTEM_PROMPT = """
You are a product design expert in a robotics company identifying new opportunities in the market for innovative consumer gadgets.

Please return a list of 10 such items in the following format:
1. Item Name
- Description
- Key Selling Points
"""

In [4]:
chat_window = [
    {
        "role": "system",
        "content": SYSTEM_PROMPT
    }
]

response = client.chat.completions.create(
    model = "gpt-4o",
    messages = chat_window,
    store = False
)
product_ideas = response.choices[0].message.content
filename = 'product_ideas.txt'
with open(filename, 'w') as file:
    file.write(product_ideas)

FileLink(filename)

/Commjhub/jupyterhub/home/ezou626/comm4190_F25_Product_Task/product_ideas.txt

In [27]:
# list products for retrieval later

pattern = re.compile(
    r"\d+\.\s+(.*?)\n- Description:\s*(.*?)\n- Key Selling Points:\s*(.*?)(?:\n|$)",
    re.DOTALL
)

def parse_product_ideas(product_list: str):
    return pattern.findall(product_list) #re.findall(r'\d+\.\s+(.*)', product_list)

parsed_product_ideas = parse_product_ideas(product_ideas)
print(parsed_product_ideas)

[('Smart Wardrobe Assistant', 'An AI-driven wardrobe organizer that suggests outfit combinations based on weather, upcoming events, and personal style preferences.', 'Customizable style algorithms, integration with smart mirrors, seasonally updated fashion tips, reduces decision fatigue.'), ('Autonomous Lawn Sculptor', 'A robotic gardener that automatically trims lawns and shapes topiary designs with precision.', 'High precision cutting, customizable topiary designs, eco-friendly operation, reduces need for professional landscaping services.'), ('Remote Pet Pal', 'An interactive pet companion that plays, feeds, and monitors pets remotely via smartphone app.', 'Treat dispensing, live video interaction, health monitoring sensors, reduces worry for busy pet owners.'), ('Sleep Optimization Robot', 'A bedside gadget that analyzes sleep patterns and dynamically adjusts room conditions (lighting, sound, temperature) for optimal rest.', 'Personalized sleep improvements, adaptive ambiance setti

In [10]:
selector_prompt_creator = lambda x: f"""
You are panelist {x} on a group of internal product evaluators that will help determine the direction a robotics company takes in investing in a new consumer gadget.
Your objective is to develop a ranking for the list of products provided to you based on your unique experience.
Please return the product numbers in a comma separated list, with most preferred product first.
"""

def get_voting_results(panelist_id):
    chat_window = [
        {
            "role": "system",
            "content": selector_prompt_creator(panelist_id)
        },
        {
            "role": "user",
            "content": "Here are the product ideas:\n" + product_ideas
        }
    ]
    
    response = client.chat.completions.create(
        model = "gpt-4o",
        messages = chat_window,
        store = False
    )
    
    return response.choices[0].message.content

results = process_map(get_voting_results, range(1, 11))

for result in results:
    print(result)

  0%|          | 0/10 [00:00<?, ?it/s]

5, 4, 6, 3, 9, 7, 2, 8, 1, 10
4, 6, 1, 9, 3, 5, 7, 10, 8, 2
4, 6, 9, 5, 3, 8, 2, 10, 1, 7
4, 3, 8, 9, 1, 6, 5, 2, 7, 10
3, 4, 7, 1, 2, 6, 5, 8, 9, 10
4, 5, 9, 3, 7, 1, 6, 10, 8, 2
4, 9, 1, 3, 5, 6, 10, 8, 2, 7
4, 9, 3, 8, 6, 5, 1, 2, 7, 10
4, 5, 3, 8, 1, 6, 9, 2, 7, 10
4, 9, 6, 3, 8, 5, 1, 10, 2, 7


In [17]:
# parse results
votes = [[int(i) for i in result.split(", ")] for result in results]

In [30]:
# evaluate ranking
from collections import Counter

def ranked_choice_ranking(votes):
    votes = [list(ballot) for ballot in votes]  # copy to avoid modifying input
    candidates = set(c for ballot in votes for c in ballot)
    elimination_order = []

    while len(candidates) > 1:
        # Count first-choice votes
        first_choices = [ballot[0] for ballot in votes if ballot]
        counts = Counter(first_choices)

        # Find lowest-vote candidate(s)
        min_votes = min(counts[c] for c in candidates)
        eliminated = [c for c in candidates if counts.get(c, 0) == min_votes]

        # Record elimination
        elimination_order.extend(eliminated)

        # Remove eliminated from ballots
        for ballot in votes:
            for e in eliminated:
                if e in ballot:
                    ballot.remove(e)

        candidates -= set(eliminated)

    # Last remaining candidate is the winner
    elimination_order.extend(candidates)

    # Reverse gives 1st → last
    return elimination_order[::-1]

ranking = ranked_choice_ranking(votes)
for i in ranking:
    name, description, _ = parsed_product_ideas[i - 1]
    print(str(i) + '.', name)
    print(description + '\n')

4. Sleep Optimization Robot
A bedside gadget that analyzes sleep patterns and dynamically adjusts room conditions (lighting, sound, temperature) for optimal rest.

5. Interactive Language Tutor
A portable AI robot that teaches languages through conversation and interactive storytelling.

3. Remote Pet Pal
An interactive pet companion that plays, feeds, and monitors pets remotely via smartphone app.

10. Augmented Reality Fashion Stylist
A wearable AR device that suggests clothing and accessory matches, visualizing them on the user through augmented reality.

9. Virtual Fitness Coach
A holographic fitness trainer that offers personalized workouts and real-time feedback in your living room.

8. Eco-Friendly Home Cleaner
An autonomous robot that uses natural, non-toxic methods for cleaning and disinfecting household surfaces.

7. AI Personal Safety Drone
A pocket-sized drone that accompanies the user to provide real-time monitoring and emergency alerts in potentially dangerous situations.

In [32]:
advancing_products = 4
products = [parsed_product_ideas[i - 1] for i in ranking[:advancing_products]]
for name, description, _ in products:
    print(name)
    print(description + '\n')

Sleep Optimization Robot
A bedside gadget that analyzes sleep patterns and dynamically adjusts room conditions (lighting, sound, temperature) for optimal rest.

Interactive Language Tutor
A portable AI robot that teaches languages through conversation and interactive storytelling.

Remote Pet Pal
An interactive pet companion that plays, feeds, and monitors pets remotely via smartphone app.

Augmented Reality Fashion Stylist
A wearable AR device that suggests clothing and accessory matches, visualizing them on the user through augmented reality.



In [46]:
# develop marketing assets (brandname, tagline)
def create_marketing_assets(data):
    product_name, product_description, company = data
    CONSULTANT_PROMPT = f"""
    You are a consultant from {company} advising a robotics in launching a new consumer gadget.
    Your objective is to develop a brand name and tag line for the new product.
    Please leverage your company's unique strengths and perspective to complete this task.
    You should format your response as "[brand name]|[tagline]"
    """

    chat_window = [
        {
            "role": "system",
            "content": CONSULTANT_PROMPT
        },
        {
            "role": "user",
            "content": f'The product is "{product_name}" and it is "{product_description}"'
        }
    ]

    response = client.chat.completions.create(
        model = "gpt-4o",
        messages = chat_window,
        store = False
    )

    return response.choices[0].message.content

marketing_consulting_companies = ["EY", "Bain"]

product_assets = []
for name, description, _ in products:
    taglines = process_map(create_marketing_assets, [(name, description, company) for company in marketing_consulting_companies])
    product_assets.append([tagline.split('|') for tagline in taglines])

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

['SomniSmart', '"Craft Your Perfect Night, One Dream at a Time"']
['HarmonyBot', '"Crafting Your Perfect Night\'s Sleep"']
A bedside gadget that analyzes sleep patterns and dynamically adjusts room conditions (lighting, sound, temperature) for optimal rest.

['LingoBot Wiz', 'Speak to Learn, Learn to Explore']
['LingoBot', '"Speak Your World, One Story at a Time"']
A portable AI robot that teaches languages through conversation and interactive storytelling.

['PetSync', '“Stay Connected, Even When Apart”']
['PetLink', '"Stay Connected, Anytime, Anywhere"']
An interactive pet companion that plays, feeds, and monitors pets remotely via smartphone app.

['StyleLens', ' Elevate Your Look, Instantly']
['StyleVista', 'Transform Your Look with a Glance']
A wearable AR device that suggests clothing and accessory matches, visualizing them on the user through augmented reality.



In [48]:
for (name, description, _), assets in zip(products, product_assets):
    print(name)
    print(description)
    for index, (brandname, tagline) in enumerate(assets):
        print(f"{index + 1}.", brandname, tagline)
    print()

Sleep Optimization Robot
A bedside gadget that analyzes sleep patterns and dynamically adjusts room conditions (lighting, sound, temperature) for optimal rest.
1. SomniSmart "Craft Your Perfect Night, One Dream at a Time"
2. HarmonyBot "Crafting Your Perfect Night's Sleep"

Interactive Language Tutor
A portable AI robot that teaches languages through conversation and interactive storytelling.
1. LingoBot Wiz Speak to Learn, Learn to Explore
2. LingoBot "Speak Your World, One Story at a Time"

Remote Pet Pal
An interactive pet companion that plays, feeds, and monitors pets remotely via smartphone app.
1. PetSync “Stay Connected, Even When Apart”
2. PetLink "Stay Connected, Anytime, Anywhere"

Augmented Reality Fashion Stylist
A wearable AR device that suggests clothing and accessory matches, visualizing them on the user through augmented reality.
1. StyleLens  Elevate Your Look, Instantly
2. StyleVista Transform Your Look with a Glance

